In [1]:
import pandas as pd
import os

Instalación de dependencias

In [2]:
# Instalamos las librerías que nos permitirá conectarse a PostgreSQL.
!pip install kaggle pandas sqlalchemy psycopg2-binary openpyxl python-dotenv

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --------------------------- ------------ 1.8/2.7 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 8.3 MB/s  0:00:00

   ------------- -------------------------- 1/3 [kagglesdk]
   ------------- -------------------------- 1/3 [kagglesdk]
   ---------------------------------------- 3/3 [kaggle]



In [ ]:
import pandas as pd
import psycopg2
import zipfile
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from sqlalchemy import create_engine
from dotenv import load_dotenv

# Cargamos las variables de entorno del archivo .env
# (Kaggle y PostgreSQL — nunca hardcodeadas en el código)
load_dotenv()

### Parte 1: Descarga mediante API de Kaggle

In [5]:
# Definimos el dataset y la carpeta de destino
dataset_name = "jenifergrategarro/dataset-public-investments-in-peru"
ruta_data = "data"

In [8]:
#Autenticamos la API
api=KaggleApi()
api.authenticate()

In [9]:
# Creamos la carpeta DATA donde se alojará la información de Kaggle
if not os.path.exists(ruta_data):
    os.makedirs(ruta_data)

In [10]:
# Descargamos y descomprimimos directamente
print(f"Descargando el dataset '{dataset_name}' de Kaggle...")
api.dataset_download_files(dataset_name, path=ruta_data, unzip=True)
print("Completado!")

Descargando el dataset 'jenifergrategarro/dataset-public-investments-in-peru' de Kaggle...
Dataset URL: https://www.kaggle.com/datasets/jenifergrategarro/dataset-public-investments-in-peru
Completado!


### Parte 2: Unificar datos 

In [11]:
ruta_data = "data"

# Leemos la descarga de Kaggle
archivos = [f for f in os.listdir(ruta_data) if f.endswith('.xlsx')]
print(f"Se encontraron {len(archivos)} archivos. Unificando...")

Se encontraron 26 archivos. Unificando...


In [ ]:
lista_dataframes = []

for archivo in archivos:
    ruta_completa = os.path.join(ruta_data, archivo)
    print(f"Procesando: {archivo}")
    df_temp = pd.read_excel(ruta_completa)
    lista_dataframes.append(df_temp)

In [17]:
# Concatenamos todos los dataframes
df_maestro = pd.concat(lista_dataframes, ignore_index=True)
print("Hecho. Unificación completada")

Hecho. Unificación completada


In [19]:
# Limpieza
df_maestro.columns = df_maestro.columns.str.lower().str.replace(' ', '_')
df_maestro = df_maestro.drop_duplicates()

print(f"\n El archivo df_maestro tiene {df_maestro.shape[0]} filas y {df_maestro.shape[1]} columnas.")


¡El archivo df_maestro tiene 381323 filas y 16 columnas.


In [20]:
# Exportamos el archivo resultante
nombre_salida = "maestro_inversiones_peru.csv"
df_maestro.to_csv(nombre_salida, index=False)
print(f"El archivo maestro se ha guardado como: '{nombre_salida}'")

El archivo maestro se ha guardado como: 'maestro_inversiones_peru.csv'


In [ ]:
# Visualizando el archivo maestro
df_maestro.head()

### Parte 3: Normalización y modelado de dimensiones

In [21]:
# Cargamos la base maestria con el nombre df
df = pd.read_csv("maestro_inversiones_peru.csv", low_memory=False)

In [22]:
# ==========================================
# TABLA 1: DIMENSIÓN GEOGRAFÍA
# ==========================================
# 1. Filtramos las columnas y quitamos vacíos
df_geografia = df[['ubigeo', 'departamento', 'provincia', 'distrito']].dropna(subset=['ubigeo'])

# 2. Eliminamos duplicados basados ÚNICAMENTE en la columna 'ubigeo'
df_geografia = df_geografia.drop_duplicates(subset=['ubigeo'], keep='first')

# 3. Renombramos y convertimos a número entero
df_geografia = df_geografia.rename(columns={'ubigeo': 'ubigeo_id', 'departamento': 'region'})
df_geografia['ubigeo_id'] = df_geografia['ubigeo_id'].astype(int)


In [23]:
# ==========================================
# TABLA 2: DIMENSIÓN SECTORES
# ==========================================
df_sectores = df[['sector']].drop_duplicates().dropna().reset_index(drop=True)

# Crear un ID numérico para cada sector (1, 2, 3...)
df_sectores['sector_id'] = df_sectores.index + 1
df_sectores = df_sectores.rename(columns={'sector': 'nombre_sector'})

In [ ]:
# ==========================================
# TABLA 3: TABLA CENTRAL PROYECTOS
# ==========================================
# Primero cruzamos el df original con nuestra nueva tabla de sectores para traernos el "sector_id"
df_cruce = df.merge(df_sectores, left_on='sector', right_on='nombre_sector', how='left')

# Seleccionamos solo las columnas necesarias (deben coincidir con las del SQL)
columnas_proyectos = [
    'código_único_de_inversión', 'nombre_de_la_inversión', 'fecha_de_registro',
    'sector_id', 'ubigeo', 'monto_viable', 'costo_actualizado', 'beneficiarios', 'estado_de_la_inversión'
]
df_proyectos = df_cruce[columnas_proyectos].drop_duplicates(subset=['código_único_de_inversión']).dropna(subset=['código_único_de_inversión'])

# Renombramos según nuestro diseño SQL
df_proyectos = df_proyectos.rename(columns={
    'código_único_de_inversión': 'proyecto_id',
    'nombre_de_la_inversión': 'nombre_proyecto',
    'fecha_de_registro': 'fecha_registro',
    'ubigeo': 'ubigeo_id',
    'monto_viable': 'costo_inicial',
    'costo_actualizado': 'costo_total',
    'estado_de_la_inversión': 'estado'
})

# Limpieza de tipos de datos
df_proyectos['proyecto_id'] = df_proyectos['proyecto_id'].astype(int)
df_proyectos['ubigeo_id'] = df_proyectos['ubigeo_id'].fillna(0).astype(int)
df_proyectos['beneficiarios'] = df_proyectos['beneficiarios'].fillna(0).astype(int)

In [25]:
# Visualizamos las tablas resultantes:
# Tabla Geografía
print("Tabla 'geografia':")
print(df_geografia.head())

Tabla 'geografia':
   ubigeo_id    region     provincia                 distrito
0      10401  AMAZONAS  CONDORCANQUI                    NIEVA
1      10000  AMAZONAS     - TODOS -                - TODOS -
3      10103  AMAZONAS   CHACHAPOYAS                   BALSAS
4      10704  AMAZONAS     UTCUBAMBA               EL MILAGRO
5      10118  AMAZONAS   CHACHAPOYAS  SAN FRANCISCO DE DAGUAS


In [26]:
# Tabla Sectores
print("\nTabla 'sectores':")
print(df_sectores.head())


Tabla 'sectores':
                  nombre_sector  sector_id
0  TRANSPORTES Y COMUNICACIONES          1
1                         SALUD          2
2           AGRICULTURA Y RIEGO          3
3          GOBIERNOS REGIONALES          4
4                     EDUCACION          5


In [27]:
# Tabla Proyectos
print("\nTabla 'proyectos':")
print(df_proyectos.head())


Tabla 'proyectos':
   proyecto_id                                    nombre_proyecto  \
0      2480997  MEJORAMIENTO DE LA CARRETERA PE - 5NC Y PE-5NE...   
1      2088578  SEGUNDA FASE DEL PROGRAMA DE APOYO A LA REFORM...   
2      2555351  MEJORAMIENTO DE LA CARRETERA PE-5NC, NUEVO SIA...   
3      2343984  ERRADICACION DE LA MOSCA DE LA FRUTA EN LOS DE...   
4      2630035  CREACION DEL SERVICIO DE TRANSITABILIDAD VIAL ...   

  fecha_registro  sector_id  ubigeo_id  costo_inicial   costo_total  estado  
0     11/02/2020          1      10401   6.104914e+08  6.562544e+08  ACTIVO  
1     21/12/2005          2      10000   4.579201e+08  4.797946e+08  ACTIVO  
2     28/06/2022          1      10401   4.122069e+08  4.551198e+08  ACTIVO  
3     06/04/2017          3      10103   4.044183e+08  4.455193e+08  ACTIVO  
4     18/01/2024          4      10704   3.728068e+08  3.728068e+08  ACTIVO  


### Parte 4: Carga a PostgreSQL

> **Prerequisito:** Antes de correr esta celda, abre pgAdmin y ejecuta la sección de creación de tablas del archivo `TRABAJO FINAL - BD INVERSION PUBLICA.sql` (líneas 1–33) para crear la base de datos `inversion_publica_db` y las tablas vacías `geografia`, `sectores` y `proyectos`.
> 
> Una vez cargados los datos, regresa al archivo SQL para ejecutar las consultas de análisis (sección 2.4).

In [28]:
from sqlalchemy import create_engine

In [ ]:
# ==========================================
# CARGA A POSTGRESQL
# ==========================================
# Ejecutamos SOLO después de haber creado la base de datos "inversion_publica_db" y las tablas de Geografia, Sectores y Proyectos con sus respectivas columnas e ID's

load_dotenv()  # Carga las variables del archivo .env

DB_USER = "postgres"
DB_PASSWORD = os.getenv('alnilam')
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "inversion_publica_db"

if not DB_PASSWORD:
    raise ValueError("No se encontró la variable 'alnilam' en el archivo .env. Verifica que el archivo .env existe y tiene la contraseña.")

cadena_conexion = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8'
engine = create_engine(cadena_conexion)

print("Subiendo a PostgreSQL...")
try:
    df_geografia.to_sql('geografia', engine, if_exists='append', index=False)
    df_sectores.to_sql('sectores', engine, if_exists='append', index=False)
    df_proyectos.to_sql('proyectos', engine, if_exists='append', index=False)
    print("Carga completada exitosamente.")
except Exception as e:
    print(f"Error en la conexión o carga: {e}")

Verificamos en SQL y empezamos con las operaciones de consulta allá.